# 🎯 Pipeline Optimizado - Datathon FME

## Objetivo
Predecir **iap_revenue_d7** (revenue de usuarios que vieron un ad) usando un modelo two-stage:
1. **Clasificador**: Predice si el usuario comprará (buyer_d7)
2. **Regresor**: Predice el revenue solo para compradores

## 📋 Instrucciones de Uso

### Ejecutar en orden:
1. **Celda 3**: Cargar datos de train con Dask
2. **Celda 5**: Procesar train (feature engineering + object parsing) → Guardar en disco
3. **Celda 7**: Preprocessing avanzado (binning, interacciones, encoding)
4. **Celda 9**: Entrenar modelo two-stage con LightGBM
5. **Celda 11**: Procesar test set
6. **Celda 12**: Generar predicciones y submission.csv

## 🔧 Preprocessing Incluido

### Fase 1: Feature Engineering Básico
- Ratios de actividad (days_active_ratio, sessions_per_week)
- Flags de comportamiento (is_new_user, is_veteran_user, recent_buyer)
- Métricas de engagement (total_time_in_app)

### Fase 2: Extracción de Object Columns
- Stats de arrays/dicts: count, sum, max, min, mean, std
- Top 7 columnas prioritarias relacionadas con revenue

### Fase 3: Preprocessing Avanzado
- **Binning**: Discretización de variables continuas (user_age, sessions, session_time)
- **Interacciones**: Features combinadas (engagement_score, avg_revenue_per_purchase)
- **Target Encoding**: Encoding con target para categóricas de baja cardinalidad
- **Frequency Encoding**: Encoding por frecuencia para top 5 categóricas

## 💾 Optimización de Memoria
- Procesamiento **lazy** con Dask (no carga todo en RAM)
- Muestreo inteligente (50% por defecto)
- Persistencia en disco (Parquet comprimido)
- RAM necesaria: **~4-8GB** (vs 26GB+ anterior)

## ⚙️ Ajustes
- Cambiar rango de fechas: Modificar `filters` en celda 3
- Ajustar muestra: Cambiar `sample_fraction` en celda 7
- Más columnas object: Editar `top_priority_cols` en celda 5
- Modelos guardados en: `./models/`
- Submission guardado en: `./outputs/submission.csv`

# 📂 PASO 1: Cargar Datos de Train

In [36]:
#!pip install dask
#!pip install pyarrow
!pip install pyarrow --upgrade

Defaulting to user installation because normal site-packages is not writeable


In [37]:
import dask
import dask.dataframe as dd

dask.config.set({"dataframe.convert-string": False})

dataset_path = "./smadex-challenge-predict-the-revenue/train/train"
filters = [("datetime", ">=", "2025-10-06-00-00"), ("datetime", "<", "2025-10-20-00-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters=filters
)

ddf = ddf.drop(columns=['buy_d14', 'buy_d28', 'buy_d7', 'buyer_d1', 'buyer_d14', 'buyer_d28', 
                        'iap_revenue_d28', 'iap_revenue_d14', 
                        'registration', 'retention_d1', 'retention_d1_to_d7', 'retention_d3',
                        'retention_d3_to_d7', 'retention_d7_to_d14', 'retentiond7'])

In [38]:
ddf

,buyer_d7,iap_revenue_d7,advertiser_bundle,advertiser_category,advertiser_subcategory,advertiser_bottom_taxonomy_level,carrier,country,region,dev_make,dev_model,dev_os,dev_osv,hour,release_date,release_msrp,weekday,avg_act_days,avg_daily_sessions,avg_days_ins,avg_duration,bcat,bcat_bottom_taxonomy,bundles_cat,bundles_cat_bottom_taxonomy,bundles_ins,city_hist,country_hist,cpm,cpm_pct_rk,ctr,ctr_pct_rk,dev_language_hist,dev_osv_hist,first_request_ts,first_request_ts_bundle,first_request_ts_category_bottom_taxonomy,hour_ratio,iap_revenue_usd_bundle,iap_revenue_usd_category,iap_revenue_usd_category_bottom_taxonomy,last_buy,last_buy_ts_bundle,last_buy_ts_category,last_ins,last_install_ts_bundle,last_install_ts_category,advertiser_actions_action_count,advertiser_actions_action_last_timestamp,user_actions_bundles_action_count,user_actions_bundles_action_last_timestamp,last_advertiser_action,new_bundles,num_buys_bundle,num_buys_category,num_buys_category_bottom_taxonomy,region_hist,rev_by_adv,rwd_prank,user_bundles,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
npartitions=24,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int32,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,int32,float64,object,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,object,object,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,int32,float64,object,object,object,object,object,category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# 🔧 PASO 2: Procesar Train y Guardar en Disco

Este paso:
- Aplica feature engineering (ratios, flags)
- Extrae features de columnas object (top 5 prioritarias)
- Guarda resultado en `./processed_train_data/`
- **NO carga todo en RAM** (procesamiento lazy)

In [39]:
# ============================================
# VERSIÓN DASK - PREPROCESSING SIN COMPUTE
# ============================================
print("=" * 80)
print("PROCESAMIENTO CON DASK (SIN CARGAR TODO EN RAM)")
print("=" * 80)

import dask.dataframe as dd
import numpy as np
import pyarrow as pa

# IMPORTANTE: NO hacer .compute() hasta el final

# ============================================
# PASO 1: FEATURE ENGINEERING EN DASK (LAZY)
# ============================================
print("\n[1/6] Feature engineering básico (lazy)...")

# 1.1 Features de ratio y proporciones
if 'avg_act_days' in ddf.columns and 'weeks_since_first_seen' in ddf.columns:
    ddf['days_active_ratio'] = ddf['avg_act_days'] / (ddf['weeks_since_first_seen'] * 7 + 1e-10)

if 'weekend_ratio' in ddf.columns:
    ddf['is_weekend_user'] = (ddf['weekend_ratio'] > 0.5).astype(int)

if 'wifi_ratio' in ddf.columns:
    ddf['is_wifi_user'] = (ddf['wifi_ratio'] > 0.7).astype(int)

# 1.2 Features de engagement temporal
if 'weeks_since_first_seen' in ddf.columns:
    ddf['user_age_days'] = ddf['weeks_since_first_seen'] * 7
    ddf['is_new_user'] = (ddf['weeks_since_first_seen'] < 1).astype(int)
    ddf['is_veteran_user'] = (ddf['weeks_since_first_seen'] > 12).astype(int)

# 1.3 Features de compra histórica
if 'last_buy' in ddf.columns:
    ddf['has_previous_purchase'] = (ddf['last_buy'] > 0).astype(int)
    ddf['days_since_last_buy'] = ddf['last_buy']
    ddf['recent_buyer'] = (ddf['last_buy'] < 7).astype(int)

# 1.4 Features de sesiones
if 'total_sessions' in ddf.columns and 'weeks_since_first_seen' in ddf.columns:
    ddf['avg_sessions_per_week'] = ddf['total_sessions'] / (ddf['weeks_since_first_seen'] + 1)

if 'avg_session_time' in ddf.columns and 'total_sessions' in ddf.columns:
    ddf['total_time_in_app'] = ddf['avg_session_time'] * ddf['total_sessions']

print("✓ 12+ features de engagement añadidas (lazy)")
print(f"✓ Particiones actuales: {ddf.npartitions}")

# ============================================
# PASO 2: PROCESAR COLUMNAS OBJECT (STATS AVANZADAS)
# ============================================
print("\n[2/6] Procesando columnas object...")

def extract_array_stats_advanced(series, prefix):
    """
    Extrae stats completas de arrays/dicts con información más rica
    """
    import ast
    import numpy as np
    import pandas as pd
    import pyarrow as pa
    
    def safe_parse(x):
        try:
            if x is None or (isinstance(x, float) and np.isnan(x)):
                return []
        except (TypeError, ValueError):
            pass
        
        try:
            parsed = ast.literal_eval(str(x))
            if isinstance(parsed, (list, dict)):
                if isinstance(parsed, dict):
                    return list(parsed.values())
                return parsed
        except:
            pass
        return []
    
    def get_numeric_values(arr):
        nums = []
        for val in arr:
            try:
                if val is not None and not (isinstance(val, float) and np.isnan(val)):
                    nums.append(float(val))
            except:
                pass
        return nums
    
    parsed = series.apply(safe_parse, meta=('x', 'object'))
    
    # Stats básicas
    count = parsed.apply(len, meta=('x', 'int64'))
    numeric_vals = parsed.apply(get_numeric_values, meta=('x', 'object'))
    
    # Stats agregadas
    def safe_sum(x): return sum(x) if x else 0
    def safe_max(x): return max(x) if x else 0
    def safe_min(x): return min(x) if x else 0
    def safe_mean(x): return np.mean(x) if x else 0
    def safe_std(x): return np.std(x) if len(x) > 1 else 0
    
    total_sum = numeric_vals.apply(safe_sum, meta=('x', 'float64'))
    total_max = numeric_vals.apply(safe_max, meta=('x', 'float64'))
    total_min = numeric_vals.apply(safe_min, meta=('x', 'float64'))
    total_mean = numeric_vals.apply(safe_mean, meta=('x', 'float64'))
    total_std = numeric_vals.apply(safe_std, meta=('x', 'float64'))
    
    return count, total_sum, total_max, total_min, total_mean, total_std

# Columnas prioritarias (revenue-related)
top_priority_cols = [
    'iap_revenue_usd_bundle',
    'num_buys_bundle',
    'whale_users_bundle_total_revenue',
    'user_bundles',
    'country_hist',
    'advertiser_actions_action_count',
    'user_actions_bundles_action_count'
]

cols_to_process = [col for col in top_priority_cols if col in ddf.columns]

print(f"Procesando {len(cols_to_process)} columnas object...")

for col in cols_to_process:
    try:
        count, total_sum, total_max, total_min, total_mean, total_std = extract_array_stats_advanced(ddf[col], col)
        
        ddf[f'{col}_count'] = count
        ddf[f'{col}_sum'] = total_sum
        ddf[f'{col}_max'] = total_max
        ddf[f'{col}_min'] = total_min
        ddf[f'{col}_mean'] = total_mean
        ddf[f'{col}_std'] = total_std
        
        print(f"  ✓ {col} → 6 features")
    except Exception as e:
        print(f"  ⚠️  {col} → Error: {str(e)[:50]}")

print(f"\n✓ Features object procesadas (lazy)")

# ============================================
# PASO 3: GUARDAR DATOS PROCESADOS EN DISCO
# ============================================
print("\n[3/6] Guardando datos procesados en disco...")

# Seleccionar solo columnas numéricas + target
numeric_cols_dask = ddf.select_dtypes(include=['int64', 'float64', 'int32', 'float32']).columns.tolist()

# Eliminar columnas object para reducir tamaño
ddf_numeric = ddf[numeric_cols_dask]

# Guardar como parquet (comprimido)
output_path = './processed_train_data'
ddf_numeric.to_parquet(output_path, compression='snappy', overwrite=True, engine='pyarrow')

print(f"✓ Datos guardados en: {output_path}")
print(f"✓ Columnas guardadas: {len(numeric_cols_dask)}")

# Liberar memoria
del ddf
import gc
gc.collect()

print("\n✓ Memoria liberada")

PROCESAMIENTO CON DASK (SIN CARGAR TODO EN RAM)

[1/6] Feature engineering básico (lazy)...
✓ 12+ features de engagement añadidas (lazy)
✓ Particiones actuales: 24

[2/6] Procesando columnas object...
Procesando 7 columnas object...
  ✓ iap_revenue_usd_bundle → 6 features
  ✓ num_buys_bundle → 6 features
  ✓ whale_users_bundle_total_revenue → 6 features
  ✓ user_bundles → 6 features
  ✓ country_hist → 6 features
  ✓ advertiser_actions_action_count → 6 features
  ✓ user_actions_bundles_action_count → 6 features

✓ Features object procesadas (lazy)

[3/6] Guardando datos procesados en disco...


NameError: name 'pa' is not defined

# 🧪 PASO 3: Preprocessing Avanzado

- Carga datos procesados (50% sample)
- **Binning**: Discretiza variables continuas
- **Interacciones**: Crea features combinadas
- **Target Encoding**: Encode categóricas con target
- **Frequency Encoding**: Encode frecuencia de categorías

In [4]:
# ============================================
# PASO 3.5: CARGAR Y APLICAR PREPROCESSING AVANZADO
# ============================================
print("\n" + "=" * 80)
print("PREPROCESSING AVANZADO (Target Encoding, Binning, Agregaciones)")
print("=" * 80)

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Cargar datos procesados
print("\n[1/5] Cargando datos procesados...")
ddf_processed = dd.read_parquet('./processed_train_data')

sample_fraction = 0.5
ddf_sample = ddf_processed.sample(frac=sample_fraction, random_state=42)
df_train = ddf_sample.compute()

print(f"✓ Datos cargados: {df_train.shape}")

# Separar targets
y_buyer = df_train['buyer_d7'] if 'buyer_d7' in df_train.columns else None
y_revenue = df_train['iap_revenue_d7'] if 'iap_revenue_d7' in df_train.columns else None
X_base = df_train.drop(columns=['buyer_d7', 'iap_revenue_d7'], errors='ignore')

# ============================================
# PREPROCESSING AVANZADO INCREMENTAL
# ============================================

# 4/6: BINNING de variables continuas
print("\n[2/5] Aplicando binning a variables continuas...")

binning_features = {}

# Binning para engagement metrics
if 'weeks_since_first_seen' in X_base.columns:
    X_base['user_age_bin'] = pd.qcut(X_base['weeks_since_first_seen'], q=5, labels=False, duplicates='drop').fillna(-1).astype(int)
    binning_features['user_age_bin'] = True

if 'total_sessions' in X_base.columns:
    X_base['sessions_bin'] = pd.qcut(X_base['total_sessions'], q=5, labels=False, duplicates='drop').fillna(-1).astype(int)
    binning_features['sessions_bin'] = True

if 'avg_session_time' in X_base.columns:
    X_base['session_time_bin'] = pd.qcut(X_base['avg_session_time'], q=5, labels=False, duplicates='drop').fillna(-1).astype(int)
    binning_features['session_time_bin'] = True

print(f"✓ {len(binning_features)} variables binned")

# 5/6: FEATURES DE INTERACCIÓN
print("\n[3/5] Creando features de interacción...")

interaction_count = 0

# Interacciones de engagement
if 'total_sessions' in X_base.columns and 'avg_session_time' in X_base.columns:
    X_base['engagement_score'] = X_base['total_sessions'] * X_base['avg_session_time']
    interaction_count += 1

if 'days_active_ratio' in X_base.columns and 'total_sessions' in X_base.columns:
    X_base['active_sessions_ratio'] = X_base['days_active_ratio'] * X_base['total_sessions']
    interaction_count += 1

# Interacciones de revenue histórico
if 'iap_revenue_usd_bundle_sum' in X_base.columns and 'num_buys_bundle_count' in X_base.columns:
    X_base['avg_revenue_per_purchase'] = X_base['iap_revenue_usd_bundle_sum'] / (X_base['num_buys_bundle_count'] + 1)
    interaction_count += 1

if 'whale_users_bundle_total_revenue_sum' in X_base.columns and 'user_bundles_count' in X_base.columns:
    X_base['whale_revenue_per_bundle'] = X_base['whale_users_bundle_total_revenue_sum'] / (X_base['user_bundles_count'] + 1)
    interaction_count += 1

print(f"✓ {interaction_count} features de interacción creadas")

# 6/6: TARGET ENCODING para categóricas (si existen)
print("\n[4/5] Aplicando target encoding a variables categóricas...")

categorical_cols = X_base.select_dtypes(include=['object', 'category']).columns.tolist()

if len(categorical_cols) > 0 and y_buyer is not None:
    from category_encoders import TargetEncoder
    
    # Solo aplicar a categóricas con cardinalidad manejable
    cat_to_encode = []
    for col in categorical_cols:
        if X_base[col].nunique() < 1000:  # Evitar alta cardinalidad
            cat_to_encode.append(col)
    
    if cat_to_encode:
        # Train/test split para evitar leakage
        X_temp_train, X_temp_val, y_temp_train, y_temp_val = train_test_split(
            X_base[cat_to_encode], y_buyer, test_size=0.2, random_state=42
        )
        
        encoder = TargetEncoder(cols=cat_to_encode, smoothing=1.0)
        encoder.fit(X_temp_train, y_temp_train)
        
        X_encoded = encoder.transform(X_base[cat_to_encode])
        X_encoded.columns = [f'{col}_target_enc' for col in cat_to_encode]
        
        X_base = pd.concat([X_base, X_encoded], axis=1)
        print(f"✓ {len(cat_to_encode)} columnas target-encoded")
else:
    print("✓ No hay columnas categóricas para encodear")

# FREQUENCY ENCODING (alternativa simple)
print("\n[5/5] Aplicando frequency encoding...")

freq_encoded = 0
for col in categorical_cols[:5]:  # Solo top 5 para evitar overhead
    if col in X_base.columns:
        freq = X_base[col].value_counts(normalize=True)
        X_base[f'{col}_freq'] = X_base[col].map(freq).fillna(0)
        freq_encoded += 1

print(f"✓ {freq_encoded} columnas frequency-encoded")

# ============================================
# RESUMEN FINAL
# ============================================
print("\n" + "=" * 80)
print("RESUMEN DE PREPROCESSING")
print("=" * 80)
print(f"Features originales:     {X_base.shape[1] - len(binning_features) - interaction_count - len(categorical_cols)*2}")
print(f"+ Binning:               {len(binning_features)}")
print(f"+ Interacciones:         {interaction_count}")
print(f"+ Target encoding:       {len(cat_to_encode) if 'cat_to_encode' in locals() else 0}")
print(f"+ Frequency encoding:    {freq_encoded}")
print(f"─" * 80)
print(f"Total features finales:  {X_base.shape[1]}")
print(f"Total samples:           {X_base.shape[0]:,}")
print("=" * 80)

# Asignar a X para compatibilidad con celdas siguientes
X = X_base


PREPROCESSING AVANZADO (Target Encoding, Binning, Agregaciones)

[1/5] Cargando datos procesados...

[1/5] Cargando datos procesados...
✓ Datos cargados: (1653239, 63)
✓ Datos cargados: (1653239, 63)

[2/5] Aplicando binning a variables continuas...
✓ 1 variables binned

[3/5] Creando features de interacción...
✓ 2 features de interacción creadas

[4/5] Aplicando target encoding a variables categóricas...
✓ No hay columnas categóricas para encodear

[5/5] Aplicando frequency encoding...
✓ 0 columnas frequency-encoded

RESUMEN DE PREPROCESSING
Features originales:     61
+ Binning:               1
+ Interacciones:         2
+ Target encoding:       0
+ Frequency encoding:    0
────────────────────────────────────────────────────────────────────────────────
Total features finales:  64
Total samples:           1,653,239

[2/5] Aplicando binning a variables continuas...
✓ 1 variables binned

[3/5] Creando features de interacción...
✓ 2 features de interacción creadas

[4/5] Aplicando targ

In [ ]:
# ============================================
# UNDER SAMPLING CON PROPORCIÓN 1:4 (nos quedamos 10% del total de datos)
# ============================================
ratio = 4
buyers = X[y_buyer == 1]
non_buyers = X[y_buyer == 0]
n_non_buyers = ratio*len(buyers)
non_buyers_sample = resample(non_buyers, n_samples = n_non_buyers, random_state = 22, replace = False)

X = pd.concat([buyers, non_buyers_sample])
y_buyer = y_buyer.loc[X.index]

In [17]:
# ============================================
# ENTRENAMIENTO TWO-STAGE CON LIGHTGBM
# ============================================
print("\n" + "=" * 80)
print("ENTRENAMIENTO DE MODELOS")
print("=" * 80)

from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error

# X, y_buyer, y_revenue ya están definidos en la celda anterior
print(f"\nDataset final:")
print(f"  Features: {X.shape[1]}")
print(f"  Samples:  {X.shape[0]:,}")

# ============================================
# TWO-STAGE MODEL: CLASIFICACIÓN + REGRESIÓN
# ============================================

# Train/test split
X_train, X_test, y_buyer_train, y_buyer_test, y_rev_train, y_rev_test = train_test_split(
    X, y_buyer, y_revenue, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_buyer
)

print(f"\nTrain: {X_train.shape[0]:,} | Test: {X_test.shape[0]:,}")

# ============================================
# ETAPA 1: CLASIFICADOR (Buyer)
# ============================================
print("\n" + "="*60)
print("ETAPA 1: CLASIFICACIÓN (buyer_d7)")
print("="*60)

params_buyer = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42,
    'scale_pos_weight': (y_buyer_train == 0).sum() / (y_buyer_train == 1).sum()
}

lgb_train = lgb.Dataset(X_train, y_buyer_train)
lgb_eval = lgb.Dataset(X_test, y_buyer_test, reference=lgb_train)

model_buyer = lgb.train(
    params_buyer,
    lgb_train,
    num_boost_round=200,
    valid_sets=[lgb_eval],
    callbacks=[lgb.early_stopping(stopping_rounds=20), lgb.log_evaluation(period=0)]
)

y_buyer_pred = model_buyer.predict(X_test, num_iteration=model_buyer.best_iteration)
auc = roc_auc_score(y_buyer_test, y_buyer_pred)

print(f"✅ AUC: {auc:.4f}")

# ============================================
# ETAPA 2: REGRESOR (Revenue)
# ============================================
print("\n" + "="*60)
print("ETAPA 2: REGRESIÓN (iap_revenue_d7)")
print("="*60)

# Solo compradores
X_train_buyers = X_train[y_buyer_train == 1]
y_rev_train_buyers = y_rev_train[y_buyer_train == 1]
# aplicamos transformación y = log(1 + y) al target
y_rev_train_buyers = np.log1p(y_rev_train_buyers)
lgb_train_rev = lgb.Dataset(X_train_buyers, y_rev_train_buyers)

print(f"Compradores en train: {len(X_train_buyers):,}")

params_revenue = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'random_state': 42
}

lgb_train_rev = lgb.Dataset(X_train_buyers, y_rev_train_buyers)

model_revenue = lgb.train(
    params_revenue,
    lgb_train_rev,
    num_boost_round=250,
    callbacks=[lgb.log_evaluation(period=0)]
)

# Predicciones combinadas
buyer_threshold = 0.3
predicted_buyers = y_buyer_pred > buyer_threshold
y_rev_pred = np.zeros(len(X_test))
y_rev_pred[predicted_buyers] = model_revenue.predict( # debido a transformación inicial predecimos expm1
    X_test[predicted_buyers], 
    num_iteration=model_revenue.best_iteration
)
y_rev_pred_test = np.zeros(len(X_test_final))
y_rev_pred_test[predicted_buyers] = np.expm1(model_revenue.predict(X_test[predicted_buyers]))

rmse = np.sqrt(mean_squared_error(y_rev_test, y_rev_pred))
mae = mean_absolute_error(y_rev_test, y_rev_pred)

print(f"✅ RMSE: ${rmse:.2f}")
print(f"✅ MAE:  ${mae:.2f}")

print("\n" + "="*60)
print("RESUMEN")
print("="*60)
print(f"AUC:  {auc:.4f}")
print(f"RMSE: ${rmse:.2f}")
print(f"MAE:  ${mae:.2f}")

# Guardar modelos
model_buyer.save_model('./models/buyer_classifier_dask.txt')
model_revenue.save_model('./models/revenue_regressor_dask.txt')
print("\n✓ Modelos guardados en ./models/")


ENTRENAMIENTO DE MODELOS

Dataset final:
  Features: 64
  Samples:  1,653,239

Train: 1,322,591 | Test: 330,648

ETAPA 1: CLASIFICACIÓN (buyer_d7)

Train: 1,322,591 | Test: 330,648

ETAPA 1: CLASIFICACIÓN (buyer_d7)
Training until validation scores don't improve for 20 rounds
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[187]	valid_0's auc: 0.722859
Did not meet early stopping. Best iteration is:
[187]	valid_0's auc: 0.722859
✅ AUC: 0.7229

ETAPA 2: REGRESIÓN (iap_revenue_d7)
✅ AUC: 0.7229

ETAPA 2: REGRESIÓN (iap_revenue_d7)
Compradores en train: 43,847
Compradores en train: 43,847
✅ RMSE: $399.62
✅ MAE:  $23.16

RESUMEN
AUC:  0.7229
RMSE: $399.62
MAE:  $23.16

✓ Modelos guardados en ./models/
✅ RMSE: $399.62
✅ MAE:  $23.16

RESUMEN
AUC:  0.7229
RMSE: $399.62
MAE:  $23.16

✓ Modelos guardados en ./models/


# 🤖 PASO 4: Entrenar Modelo Two-Stage

- Train/test split
- Entrena clasificador LightGBM (buyer_d7)
- Entrena regresor LightGBM (iap_revenue_d7)
- Guarda modelos en `./models/`

# 🎯 PASO 5: Generar Predicciones y Submission

- Carga test procesado
- Alinea features con train
- Genera predicciones
- Crea `./outputs/submission.csv`

In [18]:
# ============================================
# PROCESAR TEST SET CON DASK (VERSIÓN OPTIMIZADA)
# ============================================
print("=" * 80)
print("PROCESAMIENTO TEST SET - VERSIÓN DASK")
print("=" * 80)

# ============================================
# PASO 1: CARGAR TEST
# ============================================
print("\n[1/4] Cargando test set...")

test_path = "./smadex-challenge-predict-the-revenue/test/test"
ddf_test = dd.read_parquet(test_path)

print(f"✓ Test shape: {ddf_test.npartitions} particiones")

# IMPORTANTE: Convertir index a columna para preservar row_id
if 'row_id' not in ddf_test.columns:
    ddf_test = ddf_test.reset_index()
    if 'index' in ddf_test.columns:
        ddf_test = ddf_test.rename(columns={'index': 'row_id'})

print(f"✓ row_id preservado como columna")

# ============================================
# PASO 2: FEATURE ENGINEERING (LAZY)
# ============================================
print("\n[2/4] Feature engineering...")

if 'avg_act_days' in ddf_test.columns and 'weeks_since_first_seen' in ddf_test.columns:
    ddf_test['days_active_ratio'] = ddf_test['avg_act_days'] / (ddf_test['weeks_since_first_seen'] * 7 + 1e-10)

if 'weekend_ratio' in ddf_test.columns:
    ddf_test['is_weekend_user'] = (ddf_test['weekend_ratio'] > 0.5).astype(int)

if 'wifi_ratio' in ddf_test.columns:
    ddf_test['is_wifi_user'] = (ddf_test['wifi_ratio'] > 0.7).astype(int)

if 'last_buy' in ddf_test.columns:
    ddf_test['has_previous_purchase'] = (ddf_test['last_buy'] > 0).astype(int)

print("✓ 4 features básicas añadidas")

# ============================================
# PASO 3: PROCESAR COLUMNAS OBJECT (LAZY)
# ============================================
print("\n[3/4] Procesando columnas object...")

# Procesar solo top 5 columnas (igual que en train)
top_priority_cols_test = [
    'iap_revenue_usd_bundle',
    'num_buys_bundle',
    'whale_users_bundle_total_revenue',
    'user_bundles',
    'country_hist'
]

cols_to_process_test = [col for col in top_priority_cols_test if col in ddf_test.columns]

for col in cols_to_process_test:
    try:
        count, total_sum, total_max = extract_array_stats_simple(ddf_test[col], col)
        
        ddf_test[f'{col}_count'] = count
        ddf_test[f'{col}_sum'] = total_sum
        ddf_test[f'{col}_max'] = total_max
        
# PASO 4: SELECCIONAR FEATURES NUMÉRICAS + ROW_ID
    except Exception as e:
        print(f"  ⚠️  {col} → Error: {str(e)[:40]}")

# ============================================
# PASO 4: SELECCIONAR FEATURES NUMÉRICAS

# IMPORTANTE: Asegurar que row_id esté incluido
if 'row_id' in ddf_test.columns and 'row_id' not in numeric_cols_test:
    numeric_cols_test.append('row_id')

ddf_test_numeric = ddf_test[numeric_cols_test]

print(f"✓ row_id incluido en features guardadas")
# PASO 5: GUARDAR TEST PROCESADO
# ============================================
# PASO 5: GUARDAR TEST PROCESADO CON ROW_ID
# ============================================
print("\nGuardando test procesado...")
output_test_path = './processed_test_data'
ddf_test_numeric.to_parquet(output_test_path, compression='snappy', overwrite=True, write_index=False)

print("\n✓ Test procesado y guardado en disco")
gc.collect()
del ddf_test
# Liberar memoria
print(f"✓ Test guardado en: {output_test_path}")# Liberar memoria


PROCESAMIENTO TEST SET - VERSIÓN DASK

[1/4] Cargando test set...
✓ Test shape: 96 particiones
✓ row_id preservado como columna

[2/4] Feature engineering...
✓ 4 features básicas añadidas

[3/4] Procesando columnas object...
  ⚠️  iap_revenue_usd_bundle → Error: name 'extract_array_stats_simple' is not
  ⚠️  num_buys_bundle → Error: name 'extract_array_stats_simple' is not
  ⚠️  whale_users_bundle_total_revenue → Error: name 'extract_array_stats_simple' is not
  ⚠️  user_bundles → Error: name 'extract_array_stats_simple' is not
  ⚠️  country_hist → Error: name 'extract_array_stats_simple' is not
✓ row_id incluido en features guardadas

Guardando test procesado...
✓ Test shape: 96 particiones
✓ row_id preservado como columna

[2/4] Feature engineering...
✓ 4 features básicas añadidas

[3/4] Procesando columnas object...
  ⚠️  iap_revenue_usd_bundle → Error: name 'extract_array_stats_simple' is not
  ⚠️  num_buys_bundle → Error: name 'extract_array_stats_simple' is not
  ⚠️  whale_users_

In [ ]:
# ============================================
# GENERAR PREDICCIONES Y SUBMISSION
# ============================================
print("=" * 80)
print("GENERANDO PREDICCIONES FINALES")
print("=" * 80)

# ============================================
# PASO 1: CARGAR TEST PROCESADO
# ============================================
print("\n[1/3] Cargando test procesado...")

ddf_test_processed = dd.read_parquet('./processed_test_data')

# Computar en chunks o completo según RAM disponible
df_test = ddf_test_processed.compute()

print(f"✓ Test cargado: {df_test.shape}")

# IMPORTANTE: Extraer y preservar row_id
if 'row_id' in df_test.columns:
    row_ids = df_test['row_id'].values
    print(f"✓ row_id extraído: {len(row_ids):,} registros")
    print(f"✓ IDs únicos: {len(set(row_ids)):,}")
    
    # Verificar duplicados
    if len(row_ids) != len(set(row_ids)):
        print(f"⚠️  WARNING: Se encontraron {len(row_ids) - len(set(row_ids))} IDs duplicados")
    
    # Eliminar row_id de las features
    df_test = df_test.drop(columns=['row_id'])
else:
    print("⚠️  WARNING: No se encontró row_id en el dataset")
    row_ids = None

# ============================================
# PASO 2: ALINEAR FEATURES CON TRAIN
# ============================================
print("\n[2/3] Alineando features con train...")

# Asegurar que tenemos las mismas columnas que en train
X_test_final = pd.DataFrame(index=df_test.index)

for col in X.columns:
    if col in df_test.columns:
        X_test_final[col] = df_test[col]
    else:
        # Si falta una feature, rellenar con 0
        X_test_final[col] = 0
        print(f"  ⚠️  Missing: {col} → Filled with 0")

print(f"✓ Features alineadas: {X_test_final.shape[1]}")

# ============================================
# PASO 3: GENERAR PREDICCIONES
# ============================================
print("\n[3/3] Generando predicciones...")

# Cargar modelos guardados
model_buyer = lgb.Booster(model_file='./models/buyer_classifier_dask.txt')
model_revenue = lgb.Booster(model_file='./models/revenue_regressor_dask.txt')

# Predicción de compradores
y_buyer_pred_test = model_buyer.predict(X_test_final)

# Identificar compradores potenciales
buyer_threshold = 0.3
predicted_buyers = y_buyer_pred_test > buyer_threshold

# Predicción de revenue (solo para compradores)
y_revenue_pred_test = np.zeros(len(X_test_final))

if predicted_buyers.sum() > 0:
    y_revenue_pred_test[predicted_buyers] = model_revenue.predict(X_test_final[predicted_buyers])

# Verificar que row_ids existe y tiene el tamaño correcto
if row_ids is None:
    raise ValueError("❌ ERROR: row_ids no está disponible. Ejecuta la celda de procesamiento de test.")

if len(row_ids) != len(y_revenue_pred_test):
    raise ValueError(f"❌ ERROR: Mismatch en tamaños - row_ids: {len(row_ids)}, predicciones: {len(y_revenue_pred_test)}")

submission = pd.DataFrame({
    'row_id': row_ids,
    'iap_revenue_d7': y_revenue_pred_test
})

# Verificar duplicados antes de guardar
duplicates = submission['row_id'].duplicated().sum()
if duplicates > 0:
    print(f"⚠️  WARNING: {duplicates} IDs duplicados encontrados - eliminando duplicados...")
    submission = submission.drop_duplicates(subset=['row_id'], keep='first')

# Guardar
output_file = './outputs/submission.csv'
submission.to_csv(output_file, index=False)

print(f"\n✅ Archivo: {output_file}")
print(f"✅ Registros: {len(submission):,}")
print(f"\nPrimeras 10 predicciones:")
print(submission.head(10))

print("\n" + "=" * 80)
print("🎉 ¡LISTO PARA KAGGLE!")
print("=" * 80)

GENERANDO PREDICCIONES FINALES

[1/3] Cargando test procesado...


# 📊 CONFIGURACIÓN Y TÉCNICAS DE PREPROCESSING

## 🎨 Técnicas Implementadas

### ✅ 1. Feature Engineering Básico
```python
# Ratios y proporciones
days_active_ratio = avg_act_days / (weeks_since_first_seen * 7)
avg_sessions_per_week = total_sessions / weeks_since_first_seen

# Flags comportamentales
is_new_user = (weeks_since_first_seen < 1)
is_veteran_user = (weeks_since_first_seen > 12)
recent_buyer = (last_buy < 7)
```

### ✅ 2. Extracción de Object Columns
Convierte listas/dicts en features numéricas:
- `count`: Número de elementos
- `sum, max, min, mean, std`: Estadísticas agregadas

### ✅ 3. Binning (Discretización)
```python
# Variables continuas → Categorías
user_age_bin = pd.qcut(weeks_since_first_seen, q=5)
sessions_bin = pd.qcut(total_sessions, q=5)
session_time_bin = pd.qcut(avg_session_time, q=5)
```
**Ventaja**: Captura relaciones no lineales, reduce outliers

### ✅ 4. Features de Interacción
```python
# Combina múltiples features
engagement_score = total_sessions * avg_session_time
active_sessions_ratio = days_active_ratio * total_sessions
avg_revenue_per_purchase = iap_revenue_sum / (num_buys + 1)
```
**Ventaja**: Captura sinergias entre variables

### ✅ 5. Target Encoding
```python
# Codifica categorías con media del target
country_encoded = mean(target | country)
```
**Ventaja**: Mejor que label encoding para alta cardinalidad
**Cuidado**: Incluye smoothing y train/val split para evitar leakage

### ✅ 6. Frequency Encoding
```python
# Codifica por frecuencia de aparición
country_freq = count(country) / total_rows
```
**Ventaja**: Simple, rápido, sin leakage

## 🔧 Parámetros Configurables

### Rango de Fechas (Celda 3)
```python
# 14 días completos
filters = [("datetime", ">=", "2025-10-06-00-00"), ("datetime", "<", "2025-10-20-00-00")]

# Solo 1 día (pruebas rápidas)
filters = [("datetime", ">=", "2025-10-06-00-00"), ("datetime", "<", "2025-10-07-00-00")]
```

### Muestreo (Celda 7)
```python
sample_fraction = 0.5  # 50% (recomendado 16GB RAM)
sample_fraction = 0.8  # 80% (más datos, más RAM)
sample_fraction = 0.3  # 30% (menos memoria)
```

### Columnas Object (Celda 5)
Añade más columnas a `top_priority_cols`:
```python
top_priority_cols = [
    'iap_revenue_usd_bundle',
    'num_buys_bundle',
    'whale_users_bundle_total_revenue',
    'user_bundles',
    'country_hist',
    'advertiser_actions_action_count',  # Ya incluida
    'user_actions_bundles_action_count'  # Ya incluida
]
```

## 💡 Próximas Mejoras (Roadmap)

### 🎯 Alta Prioridad
- [ ] **RFM Features**: Recency, Frequency, Monetary para comportamiento de compra
- [ ] **Agregaciones por grupo**: Mean/max/std por country, carrier, dev_os
- [ ] **Embeddings categóricos**: Entity embeddings para alta cardinalidad
- [ ] **Time-based features**: Hour, day_of_week, is_weekend del datetime

### 🔬 Media Prioridad
- [ ] **Feature selection**: Eliminar features redundantes (correlación > 0.95)
- [ ] **Normalización**: StandardScaler para features numéricas
- [ ] **Polynomial features**: Interacciones de orden 2
- [ ] **Ensemble**: Combinar XGBoost + LightGBM + CatBoost

### ⚡ Baja Prioridad (optimización)
- [ ] **Hyperparameter tuning**: Optuna para optimizar LightGBM
- [ ] **Cross-validation**: 5-fold CV para estimación más robusta
- [ ] **Feature importance**: SHAP values para interpretabilidad

## 📁 Archivos Generados

| Archivo | Descripción |
|---------|-------------|
| `./processed_train_data/` | Train procesado (Parquet) |
| `./processed_test_data/` | Test procesado (Parquet) |
| `./models/buyer_classifier_dask.txt` | Clasificador LightGBM |
| `./models/revenue_regressor_dask.txt` | Regresor LightGBM |
| `./outputs/submission.csv` | Submission Kaggle |

## ⚠️ Troubleshooting

**Memory full**: Reducir `sample_fraction` o procesar menos object columns

**Categorical encoding error**: Verificar que las categorías existen en train y test

**Predicciones = 0**: Revisar `buyer_threshold` y que los modelos se guardaron correctamente